In [ ]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "default",
    **{
        "type": "hive",
        "uri": "thrift://hive-metastore:9083",  # Lebih rapi pakai hostname
        "s3.endpoint": "http://minio:9000",      # Lebih rapi pakai hostname
        "s3.access-key-id": "minioadmin",
        "s3.secret-access-key": "minioadmin",
        "s3.use-ssl": "false",
    }
)

In [ ]:
catalog.list_namespaces()

In [ ]:
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
arrow_table = pq.read_table("../data/yellow_tripdata_2023-01.parquet")
table_id = "test_schema.yellow_taxi"

try:
    table = catalog.create_table(
        table_id,
        schema=arrow_table.schema,
        location=f"s3a://warehouse/{table_id}"
    )
except Exception:
    table = catalog.load_table(table_id)

In [ ]:
table.append(arrow_table)
print("✅ Data taxi berhasil di-append!")

In [ ]:
# Menghapus tabel tertentu
catalog.drop_table("test_schema.yellow_taxi")

print("Tabel yellow_taxi telah dihapus dari Metastore dan file di MinIO akan dibersihkan.")

In [ ]:
catalog.drop_namespace("test_schema")
print("Namespace test_schema berhasil dihapus.")

In [2]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog(
    "default",
    **{
        "type": "hive",
        "uri": "thrift://hive-metastore:9083",  # Lebih rapi pakai hostname
        "s3.endpoint": "http://minio:9000",      # Lebih rapi pakai hostname
        "s3.access-key-id": "minioadmin",
        "s3.secret-access-key": "minioadmin",
        "s3.use-ssl": "false",
    }
)

In [ ]:
# Nama schema yang ingin dibuat
schema_name = "bronze"
# Lokasi fizikal di MinIO (pastikan diakhiri dengan /)
schema_location = "s3a://lakehouse/bronze/"


# Cara membuat namespace dengan properties location
try:
    catalog.create_namespace(
        schema_name, 
        properties={"location": schema_location}
    )
    print(f"✅ Schema '{schema_name}' berhasil dibuat di {schema_location}")
except Exception as e:
    print(f"❌ Gagal atau Schema mungkin sudah wujud: {e}")

✅ Schema 'bronze' berjaya dibuat di s3a://lakehouse/bronze/


In [4]:
import pandas as pd
import pyarrow as pa

df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")

arrow_table = pa.Table.from_pandas(df)

table_name = "yellow_taxi"
table_id = f"{schema_name}.{table_name}"  # Menjadi 'bronze.yellow_taxi'

# Lokasi fisik di MinIO
table_location = f"s3a://lakehouse/bronze/{table_name}"

In [5]:
try:
    # 1. Coba buat tabel baru
    table = catalog.create_table(
        identifier=table_id,
        schema=arrow_table.schema,
        location=table_location
    )
    print(f"✅ Tabel {table_id} berhasil dibuat baru.")
except Exception:
    # 2. Jika sudah ada, load tabel yang sudah ada
    table = catalog.load_table(table_id)
    print(f"ℹ️ Tabel {table_id} sudah ada, memuat tabel...")

# 3. Masukkan data (Append)
table.append(arrow_table)
print(f"🚀 Data berhasil dimuat ke {table_id}!")

✅ Tabel bronze.yellow_taxi berhasil dibuat baru.
🚀 Data berhasil dimuat ke bronze.yellow_taxi!


In [ ]:
# import s3fs

# # Inisialisasi koneksi ke MinIO
# fs = s3fs.S3FileSystem(
#     key='minioadmin',
#     secret='minioadmin',
#     client_kwargs={'endpoint_url': 'http://minio:9000'}
# )

# # Hapus folder yang tertinggal
# path_lama = "warehouse/test_schema.yellow_taxi"
# if fs.exists(path_lama):
#     fs.rm(path_lama, recursive=True)
#     print(f"✅ Folder {path_lama} berhasil dibersihkan dari MinIO.")